In [16]:
import os
import time
import random
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

cartoon = "cartoon_name"
front_url = "https://fanfox.net/manga/yakusoku_no_neverland/c"
# base_url = "https://fanfox.net/manga/yakusoku_no_neverland/c000/1.html#ipg{}"

print('start')

# 크롬 옵션 (헤드리스)
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0")

time.sleep(3)  # JS 로딩 대기

os.makedirs(cartoon, exist_ok=True)

for chapter in range(0, 3):
    chap_str = str(chapter).zfill(3)    
    os.makedirs(f"{cartoon}/{chap_str}", exist_ok=True)    
    
    for page in range(1, 5):
        base_url = front_url + chap_str + "/1.html#ipg" + str(page)
        url = base_url
        
        print(f"\n===== 접속: ipg : {url}") 

        driver = webdriver.Chrome(options=options)
        driver.get(url)

        sleep_time = random.uniform(1, 3)
        time.sleep(sleep_time)

        imgs = driver.find_elements(By.CLASS_NAME, "reader-main-img")

        print("찾은 이미지 개수:", len(imgs))
        if len(imgs) == 0 : break
        
        page_str = "img_" + str(page).zfill(3)
        for img in imgs:
            img_url = img.get_attribute("src")

            if img_url.startswith("//"):
                img_url = "https:" + img_url

            # print("이미지 URL:", img_url)

            # 세션 쿠키 복사
            cookies = driver.get_cookies()
            session = requests.Session()

            for cookie in cookies:
                session.cookies.set(cookie['name'], cookie['value'])

            headers = {
                "User-Agent": "Mozilla/5.0",
                "Referer": url
            }

            img_data = session.get(img_url, headers=headers).content
            filename = img_url.split("/")[-1].split("?")[0]
            
            name, ext = os.path.splitext(filename)

            with open(f"{cartoon}/{chap_str}/{page_str}{ext}", "wb") as f:
                f.write(img_data)

            print("다운로드 완료:", filename)
    
    time.sleep(sleep_time * 5)

driver.quit()
print("완료")

start

===== 접속: ipg : https://fanfox.net/manga/yakusoku_no_neverland/c000/1.html#ipg1
찾은 이미지 개수: 1
다운로드 완료: k001.jpg

===== 접속: ipg : https://fanfox.net/manga/yakusoku_no_neverland/c000/1.html#ipg2
찾은 이미지 개수: 1
다운로드 완료: k002.jpg

===== 접속: ipg : https://fanfox.net/manga/yakusoku_no_neverland/c000/1.html#ipg3
찾은 이미지 개수: 1
다운로드 완료: k003.jpg

===== 접속: ipg : https://fanfox.net/manga/yakusoku_no_neverland/c000/1.html#ipg4
찾은 이미지 개수: 1
다운로드 완료: k004.jpg

===== 접속: ipg : https://fanfox.net/manga/yakusoku_no_neverland/c001/1.html#ipg1
찾은 이미지 개수: 1
다운로드 완료: b001.jpg

===== 접속: ipg : https://fanfox.net/manga/yakusoku_no_neverland/c001/1.html#ipg2
찾은 이미지 개수: 1
다운로드 완료: b002.jpg

===== 접속: ipg : https://fanfox.net/manga/yakusoku_no_neverland/c001/1.html#ipg3
찾은 이미지 개수: 1
다운로드 완료: b003.jpg

===== 접속: ipg : https://fanfox.net/manga/yakusoku_no_neverland/c001/1.html#ipg4
찾은 이미지 개수: 1
다운로드 완료: b004.jpg

===== 접속: ipg : https://fanfox.net/manga/yakusoku_no_neverland/c002/1.html#ipg1
찾은 이미지 개수: 1


FileNotFoundError: [Errno 2] No such file or directory: 'cartoon_name/002/img_001.jpg'

000
001
002
003
004


In [11]:
front_url = "https://fanfox.net/manga/yakusoku_no_neverland/c"

In [12]:
for chapter in range(0, 3):
    chap_str = str(chapter).zfill(3)
    for page in range(5, 10):
        base_url = front_url + chap_str + "/1.html#ipg" + str(page)
        url = base_url
        print(f"\n===== 접속: ipg{page} : {url}") 
        



===== 접속: ipg5 : https://fanfox.net/manga/yakusoku_no_neverland/c000/1.html#ipg5

===== 접속: ipg6 : https://fanfox.net/manga/yakusoku_no_neverland/c000/1.html#ipg6

===== 접속: ipg7 : https://fanfox.net/manga/yakusoku_no_neverland/c000/1.html#ipg7

===== 접속: ipg8 : https://fanfox.net/manga/yakusoku_no_neverland/c000/1.html#ipg8

===== 접속: ipg9 : https://fanfox.net/manga/yakusoku_no_neverland/c000/1.html#ipg9

===== 접속: ipg5 : https://fanfox.net/manga/yakusoku_no_neverland/c001/1.html#ipg5

===== 접속: ipg6 : https://fanfox.net/manga/yakusoku_no_neverland/c001/1.html#ipg6

===== 접속: ipg7 : https://fanfox.net/manga/yakusoku_no_neverland/c001/1.html#ipg7

===== 접속: ipg8 : https://fanfox.net/manga/yakusoku_no_neverland/c001/1.html#ipg8

===== 접속: ipg9 : https://fanfox.net/manga/yakusoku_no_neverland/c001/1.html#ipg9

===== 접속: ipg5 : https://fanfox.net/manga/yakusoku_no_neverland/c002/1.html#ipg5

===== 접속: ipg6 : https://fanfox.net/manga/yakusoku_no_neverland/c002/1.html#ipg6

===== 접속: ipg7 